# Curso de Spark parte 1

In [1]:
import findspark
findspark.init()

import pandas as pd
import pyspark

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

In [3]:
spark = SparkSession.builder.getOrCreate()

In [4]:
emp = [(1, "AAA", "dept1", 1000),
    (2, "BBB", "dept1", 1100),
    (3, "CCC", "dept1", 3000),
    (4, "DDD", "dept1", 1500),
    (5, "EEE", "dept2", 8000),
    (6, "FFF", "dept2", 7200),
    (7, "GGG", "dept3", 7100),
    (8, "HHH", "dept3", 3700),
    (9, "III", "dept3", 4500),
    (10, "JJJ", "dept5", 3400)]

dept = [("dept1", "Department - 1"),
        ("dept2", "Department - 2"),
        ("dept3", "Department - 3"),
        ("dept4", "Department - 4")

       ]

df = spark.createDataFrame(emp, ["id", "name", "dept", "salary"])

deptdf = spark.createDataFrame(dept, ["id", "name"])

In [5]:
df.show()

+---+----+-----+------+
| id|name| dept|salary|
+---+----+-----+------+
|  1| AAA|dept1|  1000|
|  2| BBB|dept1|  1100|
|  3| CCC|dept1|  3000|
|  4| DDD|dept1|  1500|
|  5| EEE|dept2|  8000|
|  6| FFF|dept2|  7200|
|  7| GGG|dept3|  7100|
|  8| HHH|dept3|  3700|
|  9| III|dept3|  4500|
| 10| JJJ|dept5|  3400|
+---+----+-----+------+



In [6]:
deptdf.show()

+-----+--------------+
|   id|          name|
+-----+--------------+
|dept1|Department - 1|
|dept2|Department - 2|
|dept3|Department - 3|
|dept4|Department - 4|
+-----+--------------+



# Operaciones con los dataframes
## count: Cuenta los numeros finales

In [7]:
df.count()

10

## Columns : Muestra las columnas

In [8]:
df.columns

['id', 'name', 'dept', 'salary']

## Dtypes: Accede al dataType de columnas dentro del dataframe

In [9]:
df.dtypes

[('id', 'bigint'),
 ('name', 'string'),
 ('dept', 'string'),
 ('salary', 'bigint')]

In [10]:
# Schema: Comprueba como spark almacena el esquema del dataframe

df.schema

StructType([StructField('id', LongType(), True), StructField('name', StringType(), True), StructField('dept', StringType(), True), StructField('salary', LongType(), True)])

In [11]:
# Print Schema: Imprime lo anterior pero mas organizado

df.printSchema()

root
 |-- id: long (nullable = true)
 |-- name: string (nullable = true)
 |-- dept: string (nullable = true)
 |-- salary: long (nullable = true)



In [12]:
# Select : Selecciona columnas de un dataframe

df.select("id", "name").show()

+---+----+
| id|name|
+---+----+
|  1| AAA|
|  2| BBB|
|  3| CCC|
|  4| DDD|
|  5| EEE|
|  6| FFF|
|  7| GGG|
|  8| HHH|
|  9| III|
| 10| JJJ|
+---+----+



In [13]:
# Filter: Filtra las filas segun una condicion.
# Intentemos encontrar las filas con id= 1
# Hay diferentes formas de especificar la condicion

# Forma 1
df.filter(df["id"] == 1).show()

# Forma 2
df.filter(df.id == 1).show()

# Forma 3
df.filter(col("id")== 1).show()

# Forma 4
df.filter("id = 1").show()

+---+----+-----+------+
| id|name| dept|salary|
+---+----+-----+------+
|  1| AAA|dept1|  1000|
+---+----+-----+------+

+---+----+-----+------+
| id|name| dept|salary|
+---+----+-----+------+
|  1| AAA|dept1|  1000|
+---+----+-----+------+

+---+----+-----+------+
| id|name| dept|salary|
+---+----+-----+------+
|  1| AAA|dept1|  1000|
+---+----+-----+------+

+---+----+-----+------+
| id|name| dept|salary|
+---+----+-----+------+
|  1| AAA|dept1|  1000|
+---+----+-----+------+



In [14]:
# DROP : Elimina una columna en particular

newdf = df.drop("id")
newdf.show()

+----+-----+------+
|name| dept|salary|
+----+-----+------+
| AAA|dept1|  1000|
| BBB|dept1|  1100|
| CCC|dept1|  3000|
| DDD|dept1|  1500|
| EEE|dept2|  8000|
| FFF|dept2|  7200|
| GGG|dept3|  7100|
| HHH|dept3|  3700|
| III|dept3|  4500|
| JJJ|dept5|  3400|
+----+-----+------+



In [15]:
# Agregaciones: Podemos usar la funcion groupBy para agrupar los datos y luego "agg" para realizar la agregacion de los datos agrupados

(df.groupBy("dept")
     .agg(
         count("salary").alias("count"),
         sum("salary").alias("total"),
         min("salary").alias("minimo"),
         max("salary").alias("maximo"),
         avg("salary").alias("promedio")
     ).show()
    
)

+-----+-----+-----+------+------+--------+
| dept|count|total|minimo|maximo|promedio|
+-----+-----+-----+------+------+--------+
|dept1|    4| 6600|  1000|  3000|  1650.0|
|dept2|    2|15200|  7200|  8000|  7600.0|
|dept3|    3|15300|  3700|  7100|  5100.0|
|dept5|    1| 3400|  3400|  3400|  3400.0|
+-----+-----+-----+------+------+--------+



In [16]:
# Sorting : Sirve para ordenar una tabla de acuerdo a una columna dada

df.sort("salary").show()

+---+----+-----+------+
| id|name| dept|salary|
+---+----+-----+------+
|  1| AAA|dept1|  1000|
|  2| BBB|dept1|  1100|
|  4| DDD|dept1|  1500|
|  3| CCC|dept1|  3000|
| 10| JJJ|dept5|  3400|
|  8| HHH|dept3|  3700|
|  9| III|dept3|  4500|
|  7| GGG|dept3|  7100|
|  6| FFF|dept2|  7200|
|  5| EEE|dept2|  8000|
+---+----+-----+------+



In [17]:
# Se puede ordenar de manera desendente tambien
df.sort(desc("salary")).show(5)

+---+----+-----+------+
| id|name| dept|salary|
+---+----+-----+------+
|  5| EEE|dept2|  8000|
|  6| FFF|dept2|  7200|
|  7| GGG|dept3|  7100|
|  9| III|dept3|  4500|
|  8| HHH|dept3|  3700|
+---+----+-----+------+
only showing top 5 rows



In [18]:
# Columnas derivadas, se puede generar a partir de una funcion por columnas existentes
df.withColumn("bonus", col("salary") * .1).withColumn("total", col("bonus") + col("salary")).show()

+---+----+-----+------+-----+------+
| id|name| dept|salary|bonus| total|
+---+----+-----+------+-----+------+
|  1| AAA|dept1|  1000|100.0|1100.0|
|  2| BBB|dept1|  1100|110.0|1210.0|
|  3| CCC|dept1|  3000|300.0|3300.0|
|  4| DDD|dept1|  1500|150.0|1650.0|
|  5| EEE|dept2|  8000|800.0|8800.0|
|  6| FFF|dept2|  7200|720.0|7920.0|
|  7| GGG|dept3|  7100|710.0|7810.0|
|  8| HHH|dept3|  3700|370.0|4070.0|
|  9| III|dept3|  4500|450.0|4950.0|
| 10| JJJ|dept5|  3400|340.0|3740.0|
+---+----+-----+------+-----+------+



In [19]:
# JOINS : Se puede hacer JOINS para guardar los datos 
## Inner Join : Union total de los datos la cuales tienenm datos convinados, en este caso, no se puede agregar datos de 
df.join(deptdf, df["dept"] == deptdf["id"]).show()

+---+----+-----+------+-----+--------------+
| id|name| dept|salary|   id|          name|
+---+----+-----+------+-----+--------------+
|  1| AAA|dept1|  1000|dept1|Department - 1|
|  2| BBB|dept1|  1100|dept1|Department - 1|
|  3| CCC|dept1|  3000|dept1|Department - 1|
|  4| DDD|dept1|  1500|dept1|Department - 1|
|  5| EEE|dept2|  8000|dept2|Department - 2|
|  6| FFF|dept2|  7200|dept2|Department - 2|
|  7| GGG|dept3|  7100|dept3|Department - 3|
|  8| HHH|dept3|  3700|dept3|Department - 3|
|  9| III|dept3|  4500|dept3|Department - 3|
+---+----+-----+------+-----+--------------+



In [20]:
# Left Join: Hace un join de los datos a partir de la izquierda

df.join(deptdf,df["dept"] == deptdf["id"], "right_outer").show()

+----+----+-----+------+-----+--------------+
|  id|name| dept|salary|   id|          name|
+----+----+-----+------+-----+--------------+
|   4| DDD|dept1|  1500|dept1|Department - 1|
|   3| CCC|dept1|  3000|dept1|Department - 1|
|   2| BBB|dept1|  1100|dept1|Department - 1|
|   1| AAA|dept1|  1000|dept1|Department - 1|
|   6| FFF|dept2|  7200|dept2|Department - 2|
|   5| EEE|dept2|  8000|dept2|Department - 2|
|   9| III|dept3|  4500|dept3|Department - 3|
|   8| HHH|dept3|  3700|dept3|Department - 3|
|   7| GGG|dept3|  7100|dept3|Department - 3|
|NULL|NULL| NULL|  NULL|dept4|Department - 4|
+----+----+-----+------+-----+--------------+



In [21]:
# Right Join
df.join(deptdf, df["dept"] == deptdf["id"], "outer").show()

+----+----+-----+------+-----+--------------+
|  id|name| dept|salary|   id|          name|
+----+----+-----+------+-----+--------------+
|   1| AAA|dept1|  1000|dept1|Department - 1|
|   2| BBB|dept1|  1100|dept1|Department - 1|
|   3| CCC|dept1|  3000|dept1|Department - 1|
|   4| DDD|dept1|  1500|dept1|Department - 1|
|   5| EEE|dept2|  8000|dept2|Department - 2|
|   6| FFF|dept2|  7200|dept2|Department - 2|
|   7| GGG|dept3|  7100|dept3|Department - 3|
|   8| HHH|dept3|  3700|dept3|Department - 3|
|   9| III|dept3|  4500|dept3|Department - 3|
|NULL|NULL| NULL|  NULL|dept4|Department - 4|
|  10| JJJ|dept5|  3400| NULL|          NULL|
+----+----+-----+------+-----+--------------+



In [22]:
# Consultas SQL 

df.createOrReplaceTempView("temp_table")

spark.sql("select * from temp_table where id = 1").show()


+---+----+-----+------+
| id|name| dept|salary|
+---+----+-----+------+
|  1| AAA|dept1|  1000|
+---+----+-----+------+

